In [1]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="Mac.phil.007",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="Platform")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)


PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'Platform', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslnegotiation': 'postgres', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 



# Fecth from Apify 

HTTPS method

In [2]:
import http.client

# Initialize connection to Apify
conn = http.client.HTTPSConnection("api.apify.com")

# Set headers to request CSV format
headers = {
    'Accept': 'application/csv'
}

# Make the request (Replace with your dataset ID)
conn.request("GET", "/v2/datasets/YTeDafNhcH8ekgB5y/items?format=csv", headers=headers)

# Get response
res = conn.getresponse()
data = res.read()

# Save data as CSV file
csv_filename = "polymarket_data.csv"
with open(csv_filename, "wb") as f:
    f.write(data)

print(f"✅ CSV file saved as {csv_filename}")


✅ CSV file saved as polymarket_data.csv


In [2]:
import pandas as pd
print(pd.__version__)


2.2.3


In [3]:

csv_filename = "polymarket_data.csv"
df = pd.read_csv(csv_filename)

# Display the first few rows
df.head()


,market_chance,market_closed,market_creation_date,market_description,market_end_date,market_link,market_liquidity,market_no_price,market_num_comments,market_one_day_price_change,market_question,market_rewards_amount,market_rewards_daily_rate,market_spread,market_volume,market_volume_24,market_yes_price
0,0.001,False,2025-01-31T18:08:27.521337Z,"This market will immediately resolve to ""Yes"" ...",2025-02-28,https://polymarket.com/event/what-price-will-e...,124732.27484,0.9995,0,-0.0010,Will Ethereum hit $4750 by February 28 2025?,0.0,5.0,0.001,230183.218332,35732.109050,0.0005
1,1.000,False,2025-01-17T21:29:10.371066Z,The first round of the Romanian presidential e...,NaN,https://polymarket.com/event/romania-president...,0.00000,NaN,190,0.0000,Will Person I win the Romanian presidential el...,NaN,NaN,1.000,0.000000,0.000000,NaN
2,0.450,False,2025-02-20T23:29:27.363084Z,This market will resolve according to the numb...,2025-02-28,https://polymarket.com/event/kanye-of-tweets-f...,542.23800,0.6250,1,NaN,Will Kanye tweet 400 or more times February 20...,0.0,60.0,0.150,721.101178,721.101178,0.3750
3,1.000,False,2024-11-26T16:10:31.715428Z,Presidential elections will be held in Poland ...,NaN,https://polymarket.com/event/poland-presidenti...,0.00000,NaN,255,0.0000,Will Person H be the next President of Poland?,NaN,NaN,1.000,0.000000,0.000000,NaN
4,0.023,False,2025-01-14T20:24:19.89586Z,This market will resolve to “Yes” if the Carol...,2025-04-17,https://polymarket.com/event/presidents-trophy...,117968.91232,0.9825,1,0.0005,Will the Carolina Hurricanes win the 2025 Pres...,0.0,2.0,0.011,9390.754009,4945.000000,0.0175


In [4]:
# Check the type of every columns
df.dtypes

market_chance                  float64
market_closed                     bool
market_creation_date            object
market_description              object
market_end_date                 object
market_link                     object
market_liquidity               float64
market_no_price                float64
market_num_comments              int64
market_one_day_price_change    float64
market_question                 object
market_rewards_amount          float64
market_rewards_daily_rate      float64
market_spread                  float64
market_volume                  float64
market_volume_24               float64
market_yes_price               float64
dtype: object

# Insert into pgsql

- Don't forget to create the table first in pgsql

Create the table in pgsql

In [ ]:
# Rollback any failed transactions
connection.rollback()

# Drop the table if it already exists
cursor.execute("DROP TABLE IF EXISTS polymarket_data;")
connection.commit()

# Create a new table
create_table_query = '''CREATE TABLE polymarket_data (
    id SERIAL PRIMARY KEY,
    market_chance FLOAT8,
    market_closed BOOLEAN,
    market_creation_date TIMESTAMP,
    market_description TEXT,
    market_end_date TIMESTAMP,
    market_link TEXT,
    market_liquidity FLOAT8,
    market_no_price FLOAT8,
    market_num_comments INT,
    market_one_day_price_change FLOAT8,
    market_question TEXT,
    market_rewards_amount FLOAT8,
    market_rewards_daily_rate FLOAT8,
    market_spread FLOAT8,
    market_volume FLOAT8,
    market_volume_24 FLOAT8,
    market_yes_price FLOAT8
);'''

cursor.execute(create_table_query)
connection.commit()
print("✅ Table dropped and recreated successfully!")


In [ ]:
connection.rollback()
import pandas as pd

# Assuming df is your DataFrame
df['market_creation_date'] = df['market_creation_date'].apply(lambda x: None if pd.isna(x) else x)
df['market_end_date'] = df['market_end_date'].apply(lambda x: None if pd.isna(x) else x)

# Now insert data into PostgreSQL
for _, market in df.iterrows():
    cursor.execute("""
        INSERT INTO polymarket_data (
            market_question, market_description, market_creation_date, market_end_date, 
            market_chance, market_liquidity, market_volume, market_volume_24, 
            market_yes_price, market_no_price, market_spread, 
            market_rewards_amount, market_rewards_daily_rate, market_num_comments, market_link
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        market["market_question"], market["market_description"], 
        market["market_creation_date"], market["market_end_date"],  # Should be None if originally 'NaT'
        market["market_chance"], market["market_liquidity"], market["market_volume"], 
        market["market_volume_24"], float(market["market_yes_price"]) if market["market_yes_price"] is not None else None, 
        float(market["market_no_price"]) if market["market_no_price"] is not None else None, 
        market["market_spread"], market["market_rewards_amount"], 
        market["market_rewards_daily_rate"], market["market_num_comments"], market["market_link"]
    ))

connection.commit()
print("✅ Scraped Polymarket data stored in PostgreSQL successfully!")


In [2]:
import json
import re
import pandas as pd
import psycopg2
from flask import Flask, jsonify, render_template
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:

def clean_text(text):
    """Basic text preprocessing (lowercasing, removing special characters, stopwords)."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

def extract_topics(num_topics=3, num_words=5):
    """Perform topic modeling using NMF."""
    market_questions = get_market_questions()
    
    if not market_questions:
        return {"error": "No data available"}

    cleaned_questions = [clean_text(q) for q in market_questions]
    
    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
    X = vectorizer.fit_transform(cleaned_questions)

    # Apply NMF for topic extraction
    nmf = NMF(n_components=num_topics, random_state=42)
    W = nmf.fit_transform(X)
    H = nmf.components_

    feature_names = vectorizer.get_feature_names_out()
    topics = {}

    for i, topic in enumerate(H):
        topics[f"Topic {i+1}"] = [feature_names[j] for j in topic.argsort()[:-num_words-1:-1]]

    return topics